# Local Evaluation - Groundedness

After you have setup and configured the prompt flow, its time to evaluation its performance. Here we can use the prompt flow SDK to test different questions and see how the prompt flow performs using the evaluation prompt flows provided.

In [1]:
from promptflow import PFClient

pf_client = PFClient()

from dotenv import load_dotenv

load_dotenv()

True

In [2]:
# Add a question to test the base prompt flow.
question = "Can you tell me about your jackets?"
customerId = "4"
output = pf_client.test(
    flow="../contoso-chat", # Path to the flow directory
    inputs={ # Inputs to the flow
        "chat_history": [],
        "question": question,
        "customerId": customerId,
    },
)

output["answer"] = "".join(list(output["answer"]))

[2024-01-25 19:21:14,433][promptflow._sdk.entities._connection][WARNING] - Please use connection.secrets[key] to access secrets.


2024-01-25 19:21:14 +0000   13712 execution.flow     INFO     Start executing nodes in thread pool mode.
2024-01-25 19:21:14 +0000   13712 execution.flow     INFO     Start to run 5 nodes with concurrency level 16.
2024-01-25 19:21:14 +0000   13712 execution.flow     INFO     Executing node question_embedding. node run id: 19a4e7be-dedc-489b-8109-51e6c0d06d99_question_embedding_0
2024-01-25 19:21:14 +0000   13712 execution.flow     INFO     Executing node customer_lookup. node run id: 19a4e7be-dedc-489b-8109-51e6c0d06d99_customer_lookup_0
2024-01-25 19:21:15 +0000   13712 execution.flow     INFO     Node question_embedding completes.
2024-01-25 19:21:15 +0000   13712 execution.flow     INFO     Executing node retrieve_documentation. node run id: 19a4e7be-dedc-489b-8109-51e6c0d06d99_retrieve_documentation_0
2024-01-25 19:21:16 +0000   13712 execution.flow     INFO     Node retrieve_documentation completes.
2024-01-25 19:21:16 +0000   13712 execution.flow     INFO     Node customer_looku

In [3]:
output

{'answer': "Hello Sarah Lee! 😊 I'd be happy to tell you about our jackets. We have two fantastic options that you might love:\n\n1. Summit Breeze Jacket: 🏔️ This lightweight, windproof jacket is perfect for outdoor adventures. It's water-resistant, has adjustable cuffs, and comes with secure zippered pockets. The jacket features reflective accents for enhanced visibility during your night treks. It's a rugged yet chic choice for hiking! 🌲\n\n2. RainGuard Hiking Jacket: ☔️ This jacket is designed with waterproof, breathable fabric to keep you dry and comfortable during your outdoor undertakings. It has adjustable features, multiple pockets, and ventilation zippers for increased airflow. The reflective details make it perfect for evening treks. 🌙\n\nBased on your previous purchases, the Summit Breeze Jacket would pair well with your TrekReady Hiking Boots, and the RainGuard Hiking Jacket would complement your TrailBlaze Hiking Pants. Stay stylish and adventure-ready, Sarah Lee! 👟👖🧥",
 'c

Test the groundedness of the prompt flow with the answer from the above question.

In [4]:
test = pf_client.test(
    flow="groundedness",
    inputs={
        "question": question,
        "context": str(output["context"]),
        "answer": output["answer"],
    },
)

2024-01-25 19:21:19 +0000   13712 execution.flow     INFO     Start executing nodes in thread pool mode.
2024-01-25 19:21:19 +0000   13712 execution.flow     INFO     Start to run 2 nodes with concurrency level 16.
2024-01-25 19:21:19 +0000   13712 execution.flow     INFO     Executing node groundedness_score. node run id: 83dab139-de0a-4781-9836-882cc5f41aac_groundedness_score_0
2024-01-25 19:21:20 +0000   13712 execution.flow     INFO     Node groundedness_score completes.
2024-01-25 19:21:20 +0000   13712 execution.flow     INFO     Executing node concat_scores. node run id: 83dab139-de0a-4781-9836-882cc5f41aac_concat_scores_0
2024-01-25 19:21:20 +0000   13712 execution.flow     INFO     Node concat_scores completes.
2024-01-25 19:21:20 +0000   13712 execution.flow     INFO     Start to run 1 nodes with concurrency level 16.
2024-01-25 19:21:20 +0000   13712 execution.flow     INFO     Executing node aggregate_variants_results. node run id: 83a0b25a-f9c6-4f66-bd4b-843d60979f69_aggre

In [5]:
test

{'gpt_groundedness': 5.0}

# Local Evaluation - Multiple Metrics 

Now use the same prompt flow and test it against the Multi Evaluation flow for groundedness, coherence, fluency, and relevance.

In [6]:
test_multi = pf_client.test(
    "multi_flow",
    inputs={
        "question": question,
        "context": str(output["context"]),
        "answer": output["answer"],
    },
)

2024-01-25 19:21:20 +0000   13712 execution.flow     INFO     Start executing nodes in thread pool mode.
2024-01-25 19:21:20 +0000   13712 execution.flow     INFO     Start to run 8 nodes with concurrency level 16.
2024-01-25 19:21:20 +0000   13712 execution.flow     INFO     Executing node coherence_score. node run id: c9af746f-f6ce-4f4f-bc53-6da9f0fdddbc_coherence_score_0
2024-01-25 19:21:20 +0000   13712 execution.flow     INFO     Executing node fluency_score. node run id: c9af746f-f6ce-4f4f-bc53-6da9f0fdddbc_fluency_score_0
2024-01-25 19:21:20 +0000   13712 execution.flow     INFO     Executing node groundedness_score. node run id: c9af746f-f6ce-4f4f-bc53-6da9f0fdddbc_groundedness_score_0
2024-01-25 19:21:20 +0000   13712 execution.flow     INFO     Executing node relevance_score. node run id: c9af746f-f6ce-4f4f-bc53-6da9f0fdddbc_relevance_score_0
2024-01-25 19:21:21 +0000   13712 execution.flow     INFO     Node coherence_score completes.
2024-01-25 19:21:21 +0000   13712 executi

In [7]:
test_multi

{'gpt_coherence': 5.0,
 'gpt_fluency': 5.0,
 'gpt_groundedness': 5.0,
 'gpt_relevance': 5.0}

# AI Studio Azure batch run on an evaluation json dataset

Now in order to test these more thoroughly, we can use the Azure AI Studio to run batches of test data with the evaluation prompt flow on a larger dataset.

In [8]:
import json
# Import required libraries
from promptflow.azure import PFClient

# Import required libraries
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential

In [9]:
try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()

Populate the `config.json` file with the subscription_id, resource_group, and workspace_name.

In [10]:
config_path = "../config.json"
pf_azure_client = PFClient.from_config(credential=credential, path=config_path)

Found the config file in: ../config.json


Add the runtime from the AI Studio that will be used for the cloud batch runs.

In [11]:
# Update the runtime to the name of the runtime you created previously
runtime = "automatic"
# load flow
flow = "../contoso-chat"
# load data
data = "../data/salestestdata.jsonl"

In [12]:
# get current time stamp for run name
import datetime
now = datetime.datetime.now()
timestamp = now.strftime("%Y_%m_%d_%H%M%S")
run_name = timestamp+"chat_base_run"
print(run_name)

2024_01_25_192137chat_base_run


Create a base run to use as the variant for the evaluation runs. 

_NOTE: If you get "'An existing connection was forcibly closed by the remote host'" run the cell again._

In [13]:
# create base run in Azure Ai Studio
base_run = pf_azure_client.run(
    flow=flow,
    data=data,
    column_mapping={
        # reference data
        "customerId": "${data.customerId}",
        "question": "${data.question}",
    },
    runtime=runtime,
    # create a display name as current datetime
    display_name=run_name,
    name=run_name
)
print(base_run)

Uploading salestestdata.jsonl (< 1 MB): 100%|██████████| 465/465 [00:00<00:00, 2.94kB/s]


Uploading contoso-chat (0.68 MBs): 100%|██████████| 678978/678978 [00:01<00:00, 484734.79it/s]


[2024-01-25 19:21:42,238][promptflow][WARNING] - You're using automatic runtime, if it's first time you're using it, it may take a while to build runtime and you may see 'NotStarted' status for a while. 


Portal url: https://ai.azure.com/projectflows/bulkrun/run/2024_01_25_192137chat_base_run/details?wsid=/subscriptions/04ae8a5b-5828-493d-9ce1-8705da496495/resourcegroups/contoso-chat-rg/providers/Microsoft.MachineLearningServices/workspaces/contoso-chat-sf-aiproj
name: 2024_01_25_192137chat_base_run
created_on: '2024-01-25T19:21:46.938547+00:00'
status: NotStarted
display_name: 2024_01_25_192137chat_base_run
description:
tags: {}
properties:
  azureml.promptflow.runtime_name: automatic
  azureml.promptflow.runtime_version: 20240116.v1
  azureml.promptflow.definition_file_name: flow.dag.yaml
  azureml.promptflow.flow_lineage_id: a6117409b8b717c7e79942d8573769a6d1f4ac180e7fabad302590d2b43bff3c
  azureml.promptflow.flow_definition_datastore_name: workspaceblobstore
  azureml.promptflow.flow_definition_blob_path: LocalUpload/3580a9e4be957e1df132771a26712326/contoso-chat/flow.dag.yaml
  azureml.promptflow.inputs_mapping: '{"customerId":"${data.customerId}","question":"${data.question}"}'
  _

In [14]:
pf_azure_client.stream(base_run)

(Run status is 'NotStarted', continue streaming...)
(Run status is 'NotStarted', continue streaming...)
(Run status is 'NotStarted', continue streaming...)
(Run status is 'NotStarted', continue streaming...)
(Run status is 'NotStarted', continue streaming...)
======= Run Summary =======
Run name: "2024_01_25_192137chat_base_run"
Run status: "Failed"
Start time: "2024-01-25 19:25:13.841565+00:00"
Duration: "0:00:00"
Run url: "https://ai.azure.com/projectflows/bulkrun/run/2024_01_25_192137chat_base_run/details?wsid=/subscriptions/04ae8a5b-5828-493d-9ce1-8705da496495/resourcegroups/contoso-chat-rg/providers/Microsoft.MachineLearningServices/workspaces/contoso-chat-sf-aiproj"

InvalidRunStatusError: Failed to submit bulk run due to Automatic runtime provision failed, please check promptflow-automatic.log under current run for details.

In [ ]:
details = pf_azure_client.get_details(base_run)
details.head(10)

# Cloud Eval run on Json Data

In [ ]:
eval_flow = "multi_flow/"
data = "../data/salestestdata.jsonl"
run_name = timestamp+"chat_eval_run"
print(run_name)

eval_run_variant = pf_azure_client.run(
    flow=eval_flow,
    data=data,  # path to the data file
    run=base_run,  # use run as the variant
    column_mapping={
        # reference data
        "customerId": "${data.customerId}",
        "question": "${data.question}",
        "context": "${run.outputs.context}",
        # reference the run's output
        "answer": "${run.outputs.answer}",
    },
    runtime=runtime,
    display_name=run_name,
    name=run_name
)

In [ ]:
pf_azure_client.stream(eval_run_variant)

In [ ]:
details = pf_azure_client.get_details(eval_run_variant)
details.head(10)

In [ ]:

metrics = pf_azure_client.get_metrics(eval_run_variant)
print(json.dumps(metrics, indent=4))

In [ ]:
pf_azure_client.visualize([base_run, eval_run_variant])